In [4]:
import pandas as pd
df=pd.read_csv(r"telecom_market_data.csv")

C:\Users\MyPC\AppData\Local\Temp\ipykernel_29324\2508284979.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"telecom_market_data.csv")


In [6]:
import pandas as pd

def clean_dataset(input_filepath, output_filepath):
    """
    Cleans the telecom market dataset.

    Args:
        input_filepath (str): The path to the raw CSV file.
        output_filepath (str): The path to save the cleaned CSV file.
    """
    try:
        # Load the dataset
        print(f"Reading data from {input_filepath}...")
        df = pd.read_csv(input_filepath)
        print("Initial data loaded. Shape:", df.shape)
        print("\nInitial data types:")
        print(df.info())

        # --- Data Cleaning Steps ---

    
        # 1. Handle missing values
        # We will check for any missing values in the dataset.
        print("\nStep 2: Checking for missing values...")
        missing_values = df.isnull().sum()
        print("Missing values per column:")
        print(missing_values[missing_values > 0])
        # Based on initial inspection, the 'notes' column might be mostly empty or contain
        # non-essential data. If it has many missing values, we can drop it.
        # Let's also check columns with low variance.
        
        ''' 
       # 2. Drop uninformative columns(optional)
        print("\nStep 3: Dropping uninformative columns...")
        # Check if 'unit' or 'notes' columns have only one unique value
        cols_to_drop = []
        for col in ['unit', 'notes']:
            if col in df.columns and df[col].nunique(dropna=False) <= 1:
                cols_to_drop.append(col)
                print(f"Column '{col}' has low variance and will be dropped.")
        
        if cols_to_drop:
            df.drop(columns=cols_to_drop, inplace=True)
        '''
        # 3. Remove duplicate rows
        print("\nStep 4: Removing duplicate rows...")
        initial_rows = len(df)
        df.drop_duplicates(inplace=True)
        final_rows = len(df)
        print(f"Removed {initial_rows - final_rows} duplicate rows.")

        # 4. Clean text data
        # Trimming leading/trailing whitespace from object columns
        print("\nStep 5: Cleaning and standardizing text data...")
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].str.strip()
        print("Trimmed whitespace from text columns.")


        # 5. Optimize data types
        # Convert columns with a limited number of unique values to 'category' type to save memory.
        print("\nStep 6: Optimizing data types...")
        for col in ['circle', 'type_of_connection', 'service_provider']:
             if col in df.columns:
                df[col] = df[col].astype('category')
        
        print("\nFinal cleaned data info:")
        df.info()

        # Reorder columns for better readability
        if 'date' in df.columns:
            cols = df.columns.tolist()
            # Bring date to the front
            cols.insert(0, cols.pop(cols.index('date')))
            df = df[cols]

        # Save the cleaned dataframe to a new CSV file
        print(f"\nSaving cleaned data to {output_filepath}...")
        df.to_csv(output_filepath, index=False)
        print("Done! Your dataset has been cleaned successfully.")

    except FileNotFoundError:
        print(f"Error: The file {input_filepath} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


if __name__ == '__main__':
    # Define the input and output file paths
    # The script expects 'telecom_market_data.csv' to be in the same directory.
    INPUT_FILE = 'telecom_market_data.csv'
    OUTPUT_FILE = 'semi_cleaned_telecom_data.csv'
    
    # Run the cleaning function
    clean_dataset(INPUT_FILE, OUTPUT_FILE)


Reading data from telecom_market_data.csv...
Initial data loaded. Shape: (70728, 8)

Initial data types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70728 entries, 0 to 70727
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   year                70728 non-null  int64 
 1   month               70728 non-null  object
 2   circle              70728 non-null  object
 3   type_of_connection  70728 non-null  object
 4   service_provider    70728 non-null  object
 5   value               58396 non-null  object
 6   unit                70728 non-null  object
 7   notes               511 non-null    object
dtypes: int64(1), object(7)
memory usage: 4.3+ MB
None

Step 2: Checking for missing values...
Missing values per column:
value    12332
notes    70217
dtype: int64

Step 4: Removing duplicate rows...
Removed 0 duplicate rows.

Step 5: Cleaning and standardizing text data...


C:\Users\MyPC\AppData\Local\Temp\ipykernel_29324\846736545.py:14: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_filepath)


Trimmed whitespace from text columns.

Step 6: Optimizing data types...

Final cleaned data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70728 entries, 0 to 70727
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   year                70728 non-null  int64   
 1   month               70728 non-null  object  
 2   circle              70728 non-null  category
 3   type_of_connection  70728 non-null  category
 4   service_provider    70728 non-null  category
 5   value               53496 non-null  object  
 6   unit                70728 non-null  object  
 7   notes               511 non-null    object  
dtypes: category(3), int64(1), object(4)
memory usage: 2.9+ MB

Saving cleaned data to semi_cleaned_telecom_data.csv...
Done! Your dataset has been cleaned successfully.


In [8]:
import pandas as pd
df=pd.read_csv(r"semi_cleaned_telecom_data.csv")

C:\Users\MyPC\AppData\Local\Temp\ipykernel_29324\2783226207.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"semi_cleaned_telecom_data.csv")


In [9]:
df.dtypes

year                   int64
month                 object
circle                object
type_of_connection    object
service_provider      object
value                 object
unit                  object
notes                 object
dtype: object

In [10]:
import numpy as np

# 1. Replace explicit empty strings ("") AND single-space strings (' ') with 0
df['value'] = df['value'].replace(['', ' '], 0, regex=False)

# 2. Convert the 'value' column to numeric. 'coerce' converts any remaining messy/non-numeric strings to NaN.
df['value'] = pd.to_numeric(df['value'], errors='coerce')

# 3. Fill any NaN values (either original missing or from step 2) with 0
df['value'] = df['value'].fillna(0)

# 4. Finally, ensure the entire column is an integer type (if appropriate for your data)
df['value'] = df['value'].astype(int)
df['value']

0        33965795
1        12314102
2        40967773
3        18877637
4        12401101
           ...   
70723           0
70724           0
70725           0
70726           0
70727           0
Name: value, Length: 70728, dtype: int32

In [11]:
df.dtypes

year                   int64
month                 object
circle                object
type_of_connection    object
service_provider      object
value                  int32
unit                  object
notes                 object
dtype: object

In [ ]:
#update the datacleaned dataset
df.to_csv("cleaned_telecom_data.csv",index=False)
print("Data saved to cleaned_telecom_data.csv")

Total rows: 70728
Data saved to cleaned_telecom_data.csv


In [13]:
df.head()

,year,month,circle,type_of_connection,service_provider,value,unit,notes
0,2025,April,Andhra Pradesh,wireless,Bharti Airtel (Including Tata Tele.),33965795,value in absolute number,NaN
1,2025,April,Assam,wireless,Bharti Airtel (Including Tata Tele.),12314102,value in absolute number,NaN
2,2025,April,Bihar,wireless,Bharti Airtel (Including Tata Tele.),40967773,value in absolute number,NaN
3,2025,April,Delhi,wireless,Bharti Airtel (Including Tata Tele.),18877637,value in absolute number,NaN
4,2025,April,Gujarat,wireless,Bharti Airtel (Including Tata Tele.),12401101,value in absolute number,NaN


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70728 entries, 0 to 70727
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   year                70728 non-null  int64 
 1   month               70728 non-null  object
 2   circle              70728 non-null  object
 3   type_of_connection  70728 non-null  object
 4   service_provider    70728 non-null  object
 5   value               70728 non-null  int32 
 6   unit                70728 non-null  object
 7   notes               511 non-null    object
dtypes: int32(1), int64(1), object(6)
memory usage: 4.0+ MB


In [15]:
a=df['service_provider'].unique()
c=0
for i in a:
    print(i)
    c+=1
a 
print('Number of unique service providers after cleaning: ',c)

Bharti Airtel (Including Tata Tele.)
Reliance Communications
Vodafone Idea
BSNL
MTNL
Reliance Jio
BSNL (VNOs)
Bharti Airtel
Tata Teleservices
Vodafone
Idea
Telenor
Aircel
Sistema
Quadrant
Videocon
Telewings
Loop
Unitech
Aircel/Dishnet
Quadrant (HFCL)
S-Tel
Etisalat
Uninor
HFCL
Etisalat/Allianz
Idea/Spice
Vodafone Essar
Loop Telecom Pvt. Ltd.
HFCL Infotel
Sistema Shyam Teleservices Ltd
Spice
BPL Mobile
Tata Telecommunication
STPL
APSFL
V-CON Mobile & Infra Pvt. Ltd.
Bharti Airtel (including Tata Telecommunications)
Reliance
Tata
Teleservice Ltd.
Systema Shyam Teleservices Ltd.
Number of unique service providers after cleaning:  42


In [16]:
a=df['circle'].unique()
a 

array(['Andhra Pradesh', 'Assam', 'Bihar', 'Delhi', 'Gujarat', 'Haryana',
       'Himachal Pradesh', 'Jammu and Kashmir', 'Karnataka', 'Kerala',
       'Kolkata', 'Madhya Pradesh', 'Maharashtra', 'Mumbai', 'North East',
       'Odisha', 'Punjab', 'Rajasthan', 'Tamil Nadu',
       'Uttar Pradesh (East)', 'Uttar Pradesh (West)', 'West Bengal',
       'All India', 'All india', 'Tamil Nadu (including Chennai)',
       'Chennai', 'Andaman and Nicobar Islands', 'Arunachal Pradesh',
       'Chhattisgarh', 'Jharkhand', 'Uttarakhand', 'North East 1',
       'North East 2', 'North East2', 'North East1', 'Chattisgarh',
       'Andaman and Nicobar', 'Uttaranchal'], dtype=object)

In [11]:
c=0
for i in a:
    print(i,type(i))
    c+=1
print('Number of unique circle after cleaning:',c)

Andhra Pradesh <class 'str'>
Assam <class 'str'>
Bihar <class 'str'>
Delhi <class 'str'>
Gujarat <class 'str'>
Haryana <class 'str'>
Himachal Pradesh <class 'str'>
Jammu and Kashmir <class 'str'>
Karnataka <class 'str'>
Kerala <class 'str'>
Kolkata <class 'str'>
Madhya Pradesh <class 'str'>
Maharashtra <class 'str'>
Mumbai <class 'str'>
North East <class 'str'>
Odisha <class 'str'>
Punjab <class 'str'>
Rajasthan <class 'str'>
Tamil Nadu <class 'str'>
Uttar Pradesh (East) <class 'str'>
Uttar Pradesh (West) <class 'str'>
West Bengal <class 'str'>
All India <class 'str'>
All india <class 'str'>
Tamil Nadu (including Chennai) <class 'str'>
Chennai <class 'str'>
Andaman and Nicobar Islands <class 'str'>
Arunachal Pradesh <class 'str'>
Chhattisgarh <class 'str'>
Jharkhand <class 'str'>
Uttarakhand <class 'str'>
North East 1 <class 'str'>
North East 2 <class 'str'>
North East2 <class 'str'>
North East1 <class 'str'>
Chattisgarh <class 'str'>
Andaman and Nicobar <class 'str'>
Uttaranchal <clas

In [12]:
a=df['year'].unique()
c=0
for i in a:
    print(i,type(i))
    c+=1
print('Number of unique years after cleaning:',c)

2025 <class 'numpy.int64'>
2024 <class 'numpy.int64'>
2023 <class 'numpy.int64'>
2022 <class 'numpy.int64'>
2021 <class 'numpy.int64'>
2020 <class 'numpy.int64'>
2019 <class 'numpy.int64'>
2018 <class 'numpy.int64'>
2017 <class 'numpy.int64'>
2016 <class 'numpy.int64'>
2015 <class 'numpy.int64'>
2014 <class 'numpy.int64'>
2013 <class 'numpy.int64'>
2012 <class 'numpy.int64'>
2011 <class 'numpy.int64'>
2010 <class 'numpy.int64'>
2009 <class 'numpy.int64'>
Number of unique years after cleaning: 17


In [13]:
c=0
for i in df['value']:
    if i==0:
        c+=1
print(c)

22654
